In [8]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(".."))
from src.sentiment_analysis import analyze_sentiment
from src.theme_extraction import extract_keywords
from src.theme_clustering import cluster_keywords_to_themes
from src.merge_sentiment import merge_sentiment_with_reviews

In [9]:
# Step 1: Sentiment Analysis
analyze_sentiment(
    input_path="../data/cleaned/all_banks_reviews_clean.csv",
    output_path="../outputs/sentiment/all_banks_sentiment.csv"
)

Device set to use cuda:0


 Sentiment saved to ../outputs/sentiment/all_banks_sentiment.csv


,review,rating,date,bank,source,sentiment_label,sentiment_score
0,good,4,2025-06-05,Commercial Bank of Ethiopia,Google Play,POSITIVE,0.999816
1,not functional,1,2025-06-05,Commercial Bank of Ethiopia,Google Play,NEGATIVE,0.999779
2,everytime you uninstall the app you have to re...,1,2025-06-04,Commercial Bank of Ethiopia,Google Play,NEGATIVE,0.990516
3,አካውንት የምናስገባበት ቦታ ስም ጽፈን ነው ከዚህ በፊት የላክንባቸውን አ...,4,2025-06-04,Commercial Bank of Ethiopia,Google Play,NEGATIVE,0.961685
4,best,5,2025-06-04,Commercial Bank of Ethiopia,Google Play,POSITIVE,0.999794
...,...,...,...,...,...,...,...
1453,very good bank in the world Digital technology...,5,2021-08-24,Dashen Bank,Google Play,POSITIVE,0.998812
1454,"App is very slow, and not logged in for so man...",1,2021-08-24,Dashen Bank,Google Play,NEGATIVE,0.999596
1455,very slowly working app,3,2021-08-22,Dashen Bank,Google Play,NEGATIVE,0.998403
1456,easy transfer of funds. user-friendly,5,2021-08-19,Dashen Bank,Google Play,POSITIVE,0.991739


In [10]:
# Step 2: Merge Sentiment + Review
merge_sentiment_with_reviews(
    sentiment_path="../outputs/sentiment/all_banks_sentiment.csv",
    output_path="../outputs/sentiment/reviews_with_sentiment.csv"
)

 Merged sentiment with review data at: ../outputs/sentiment/reviews_with_sentiment.csv


,review,rating,date,bank,sentiment_label,sentiment_score
0,good,4,2025-06-05,Commercial Bank of Ethiopia,POSITIVE,0.999816
1,not functional,1,2025-06-05,Commercial Bank of Ethiopia,NEGATIVE,0.999779
2,everytime you uninstall the app you have to re...,1,2025-06-04,Commercial Bank of Ethiopia,NEGATIVE,0.990516
3,አካውንት የምናስገባበት ቦታ ስም ጽፈን ነው ከዚህ በፊት የላክንባቸውን አ...,4,2025-06-04,Commercial Bank of Ethiopia,NEGATIVE,0.961685
4,best,5,2025-06-04,Commercial Bank of Ethiopia,POSITIVE,0.999794
...,...,...,...,...,...,...
1453,very good bank in the world Digital technology...,5,2021-08-24,Dashen Bank,POSITIVE,0.998812
1454,"App is very slow, and not logged in for so man...",1,2021-08-24,Dashen Bank,NEGATIVE,0.999596
1455,very slowly working app,3,2021-08-22,Dashen Bank,NEGATIVE,0.998403
1456,easy transfer of funds. user-friendly,5,2021-08-19,Dashen Bank,POSITIVE,0.991739


In [11]:
# Step 3: Extract Keywords
extract_keywords(
    input_path="../outputs/sentiment/all_banks_sentiment.csv",
    output_path="../outputs/themes/bank_keywords.csv",
    top_n=50
)

 Keyword output saved to ../outputs/themes/bank_keywords.csv


,bank,top_keywords
0,Commercial Bank of Ethiopia,"app, good, best, nice, cbe, like, great, bank,..."
1,Bank of Abyssinia,"app, good, work, bank, worst, working, doesn, ..."
2,Dashen Bank,"good, app, bank, service, doha, working, excel..."


In [12]:
# Step 4: Cluster Keywords to Themes
THEME_RULES = {
    "Login & Access": ["login", "password", "otp", "access", "pin", "credentials", "account"],
    "Performance": ["slow", "crash", "freeze", "lag", "loading", "hang", "bug"],
    "UI/UX": ["ui", "interface", "design", "navigation", "layout", "display", "theme"],
    "Feature Request": ["feature", "add", "option", "budget", "history", "notification", "statement"],
    "Customer Support": ["support", "help", "service", "contact", "response", "call", "assist"]
}
cluster_keywords_to_themes(
    input_path="../outputs/themes/bank_keywords.csv",
    output_path="../outputs/themes/bank_themes_final.csv",
    theme_rules=THEME_RULES
)

 Clustered themes saved to ../outputs/themes/bank_themes_final.csv


,bank,theme,keywords
0,Commercial Bank of Ethiopia,Customer Support,service
1,Bank of Abyssinia,Performance,slow
2,Bank of Abyssinia,Feature Request,option
3,Bank of Abyssinia,Customer Support,service
4,Dashen Bank,Login & Access,"login, account"
5,Dashen Bank,Performance,slow
6,Dashen Bank,Customer Support,service


In [13]:
# Step 5: Aggregate Sentiment by Bank and Rating
import pandas as pd

df = pd.read_csv("../outputs/sentiment/all_banks_sentiment.csv")
summary = df.groupby(["bank", "rating"])["sentiment_score"].mean().reset_index()
summary.to_csv("../outputs/sentiment/sentiment_by_bank_and_rating.csv", index=False)
print(" Sentiment summary saved to: ../outputs/sentiment/sentiment_by_bank_and_rating.csv")


 Sentiment summary saved to: ../outputs/sentiment/sentiment_by_bank_and_rating.csv
